In [1]:

from keras import backend as K
K.tensorflow_backend._get_available_gpus()
# import resnet50
%matplotlib inline


Using TensorFlow backend.


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import json
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
from math import sqrt
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder, PolynomialFeatures
import glob
from joblib import Parallel, delayed

# IMPORT DATA

In [3]:
# Change directory
path = "C:/Users/cptien/Desktop/data/pet_adoption_data/"
os.chdir(path)


In [4]:
os.listdir('./')

['breed_labels.csv',
 'color_labels.csv',
 'sample_submission.csv',
 'state_labels.csv',
 'test.csv',
 'test.zip',
 'test_images',
 'test_images.zip',
 'test_metadata',
 'test_metadata.zip',
 'test_sentiment',
 'test_sentiment.zip',
 'train.csv',
 'train.zip',
 'train_images',
 'train_metadata',
 'train_sentiment']

In [5]:
train = pd.read_csv("./train.csv")
test  = pd.read_csv("./test.csv")
sample_submission = pd.read_csv("./sample_submission.csv")
breed_labels = pd.read_csv("./breed_labels.csv")
color_labels = pd.read_csv("./color_labels.csv")
state_labels = pd.read_csv("./state_labels.csv")

## Load Image, Metadata, and Sentiment

In [6]:
train_img_files = sorted(glob.glob('./train_images/*.jpg'))
train_mdata_files = sorted(glob.glob('./train_metadata/*.json'))
train_senti_files = sorted(glob.glob('./train_sentiment/*.json'))
print(f'Number of train image files: {len(train_img_files)}')
print(f'Number of train metadata files: {len(train_mdata_files)}')
print(f'Number of train sentiment files: {len(train_senti_files)}')

Number of train image files: 58311
Number of train metadata files: 58311
Number of train sentiment files: 14442


In [7]:
test_img_files = sorted(glob.glob('./test_images/*.jpg'))
test_mdata_files = sorted(glob.glob('./test_metadata/*.json'))
test_senti_files = sorted(glob.glob('./test_sentiment/*.json'))
print(f'Number of test image files: {len(test_img_files)}')
print(f'Number of test metadata files: {len(test_mdata_files)}')
print(f'Number of test sentiment files: {len(test_senti_files)}')

Number of test image files: 14465
Number of test metadata files: 14465
Number of test sentiment files: 3865


## Train Data

In [8]:
#images
train_ids = train[['PetID']]
print(f'Total pets: {len(train_ids)}')
train_imgs = pd.DataFrame(train_img_files)
train_imgs.columns = ['image_filenames']
train_imgs_pets = train_imgs['image_filenames'].apply(lambda x: x.split('\\')[-1].split('-')[0])
train_imgs = train_imgs.assign(PetID=train_imgs_pets)
print(f'Pets with images: {len(train_imgs_pets.unique())}')
pets_with_images = len(np.intersect1d(train_imgs_pets.unique(), 
                                      train_ids['PetID'].unique()))
print(f'Fraction of pets with images: {pets_with_images/train_ids.shape[0]:.3f}')

#metadata
print()
train_mdata = pd.DataFrame(train_mdata_files, columns=['metadata_filenames'])
train_mdata_pets = train_mdata['metadata_filenames'].apply(lambda x:x.split('\\')[-1].split('-')[0])
train_mdata['PetID'] = train_mdata_pets
print(f'Pets with Metadata: {len(train_mdata_pets.unique())}')
pets_with_metadata = len(np.intersect1d(train_mdata_pets.unique(), train_ids['PetID'].unique()))
print(f'Fraction of pets with Metadata: {pets_with_metadata/train_ids.shape[0]:.3}')
print()

#sentiments
train_senti = pd.DataFrame(train_senti_files, columns=['sentiment_filenames'])
train_senti_pets = train_senti['sentiment_filenames'].apply(lambda x: x.split('\\')[-1].split('.')[0])
train_senti['PetID'] = train_senti_pets
print(f'Pets with Sentiments: {len(train_senti_pets.unique())}')

pets_with_senti = len(np.intersect1d(train_senti_pets.unique(), train_ids['PetID'].unique()))
print(f'Fraction of pets with sentiments: {pets_with_senti/train_ids.shape[0]:.3}')

Total pets: 14993
Pets with images: 14652
Fraction of pets with images: 0.977

Pets with Metadata: 14652
Fraction of pets with Metadata: 0.977

Pets with Sentiments: 14442
Fraction of pets with sentiments: 0.963


## Test Data

In [9]:
#images
test_ids = test[['PetID']]
print(f'Total pets: {len(test_ids)}')
test_imgs = pd.DataFrame(test_img_files,columns=['image_filenames'])

test_imgs_pets = test_imgs['image_filenames'].apply(lambda x: x.split('\\')[-1].split('-')[0])
test_imgs = test_imgs.assign(PetID=test_imgs_pets)
print(f'Pets with images: {len(test_imgs_pets.unique())}')
pets_with_images = len(np.intersect1d(test_imgs_pets.unique(), 
                                      test_ids['PetID'].unique()))
print(f'Fraction of pets with images: {pets_with_images/test_ids.shape[0]:.3f}')

#metadata
print()
test_mdata = pd.DataFrame(test_mdata_files, columns=['metadata_filenames'])
test_mdata_pets = test_mdata['metadata_filenames'].apply(lambda x:x.split('\\')[-1].split('-')[0])
test_mdata['PetID'] = test_mdata_pets
print(f'Pets with Metadata: {len(test_mdata_pets.unique())}')
pets_with_metadata = len(np.intersect1d(test_mdata_pets.unique(), test_ids['PetID'].unique()))
print(f'Fraction of pets with Metadata: {pets_with_metadata/test_ids.shape[0]:.3}')
print()

#sentiments
test_senti = pd.DataFrame(test_senti_files, columns=['sentiment_filenames'])
test_senti_pets = test_senti['sentiment_filenames'].apply(lambda x: x.split('\\')[-1].split('.')[0])
test_senti['PetID'] = test_senti_pets
print(f'Pets with Sentiments: {len(test_senti_pets.unique())}')

pets_with_senti = len(np.intersect1d(test_senti_pets.unique(), test_ids['PetID'].unique()))
print(f'Fraction of pets with sentiments: {pets_with_senti/test_ids.shape[0]:.3}')
print()
print(f'Images and Metadata distribution are the same? {np.all(test_mdata_pets==test_imgs_pets)}')

Total pets: 3972
Pets with images: 3858
Fraction of pets with images: 0.971

Pets with Metadata: 3858
Fraction of pets with Metadata: 0.971

Pets with Sentiments: 3865
Fraction of pets with sentiments: 0.973

Images and Metadata distribution are the same? True


In [10]:
from abc import ABCMeta, abstractmethod


In [11]:
class PetFinderParser(object):
    def __init__(self, debug=False):
        self.debug = debug
        self.sentence_sep = ' '
        self.extract_sentiment_text = False
    
    def open_metadata_file(self, filename):
        # Load metadata file
        with open(filename, 'r', encoding='utf-8') as f:
            metadata_file = json.load(f)
        return metadata_file
    
    def open_sentiment_file(self, filename):
        # Load sentiment file
        with open(filename, 'r', encoding='utf-8') as f:
            sentiment_file = json.load(f)
        return sentiment_file
    
    def open_image_file(self, filename):
        # Load image file
        with open(filename, 'r') as f:
            image_file = json.load(f)
        return image_file

    def parse_sentiment_file(self, file):
        # Parse sentiment file.
        file_sentiment = file['documentSentiment']
        file_entities = [x['name'] for x in file['entities']]
        file_entities = self.sentence_sep.join(file_entities)
        
        if self.extract_sentiment_text:
            file_sentence_text = [x['text']['content'] for x in file['sentences']]
            file_sentence_text = self.sentence_sep.join(file_sentence_text)
            
        file_sentence_sentiment = [x['sentiment'] for x in file['sentences']]
        file_sentence_sentiment = pd.DataFrame.from_dict(file_sentence_sentiment, orient='columns').sum()
        file_sentence_sentiment = file_sentence_sentiment.add_prefix('document_').to_dict()
        
        file_sentiment.update(file_sentence_sentiment)
        
        df_sentiment = pd.DataFrame.from_dict(file_sentiment, orient='index').T
        if self.extract_sentiment_text:
            df_sentiment['text'] = file_sentence_text
        
        df_sentiment['entities'] = file_entities
        df_sentiment = df_sentiment.add_prefix('sentiment_')
        
        return df_sentiment
    
    def parse_metadata_file(self, file):
        # Parsse metadta file
        
        file_keys=list(file.keys())
        
        if 'labelAnnotations' in file_keys:
            file_annots = file['labelAnnotations'][:int(len(file['labelAnnotations']) * 0.3)]
            file_top_score = np.asarray([x['score'] for x in file_annots]).mean()
            file_top_desc = [x['description'] for x in file_annots]
        else:
            file_top_score = np.nan
            file_top_desc = ['']
            
        file_colors = file['imagePropertiesAnnotation']['dominantColors']['colors']
        file_crops = file['cropHintsAnnotation']['cropHints']
        
        file_color_score = np.asarray([x['score'] for x in file_colors]).mean()
        file_color_pixelfrac = np.asarray([x['pixelFraction'] for x in file_colors]).mean()
        
        file_crop_conf = np.asarray([x['confidence'] for x in file_crops]).mean()
        
        if 'importanceFraction'  in file_crops[0].keys():
            file_crop_importance = np.asarray([x['importanceFraction'] for x in file_crops]).mean()
        
        else:
            file_crop_importance = np.nan
            
        df_metadata = {
            'annots_score': file_top_score,
            'color_score': file_color_score,
            'color_pixelfrac': file_color_pixelfrac,
            'crop_conf': file_crop_conf,
            'crop_importance': file_crop_importance,
            'annots_top_desc': self.sentence_sep.join(file_top_desc)
        }
            
        df_metadata = pd.DataFrame.from_dict(df_metadata, orient='index').T
        df_metadata = df_metadata.add_prefix('metadata_')
        
        return df_metadata
    
# Helper fuction for parallel data processing:
def extract_additional_features(pet_id, mode='train'):
    sentiment_filename = f'./{mode}_sentiment/{pet_id}.json'
    
    try:
        sentiment_file = pet_parser.open_sentiment_file(sentiment_filename)
        df_sentiment = pet_parser.parse_sentiment_file(sentiment_file)
        df_sentiment['PetID'] = pet_id
    except FileNotFoundError:
        df_sentiment = []
    
    dfs_metadata = []
    metadata_filenames = sorted(glob.glob(f'./{mode}_metadata/{pet_id}*.json'))
    if len(metadata_filenames) > 0:
        for f in metadata_filenames:
            metadata_file = pet_parser.open_metadata_file(f)
            df_metadata = pet_parser.parse_metadata_file(metadata_file)
            df_metadata['PetID'] = pet_id
            dfs_metadata.append(df_metadata)
        dfs_metadata = pd.concat(dfs_metadata, ignore_index=True, sort=False)
    dfs= [df_sentiment, dfs_metadata]
        
    return dfs
            
            
pet_parser = PetFinderParser()

In [12]:
# Unique IDs from train and test:
debug = False
train_pet_ids = train['PetID'].unique()
test_pet_ids = test['PetID'].unique()

if debug:
    train_pet_ids = train_pet_ids[:1000]
    test_pet_ids = test_pet_ids[:500]
    

In [13]:
# Train Set:
# Parallel processsing of data:
dfs_train = Parallel(n_jobs=6, verbose=1)(delayed(extract_additional_features)(i,mode='train') for i in train_pet_ids)
# dfs_train = [extract_additional_features(i, mode='train') for i in train_pet_ids]


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:   16.1s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:   32.5s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:  1.0min
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:  1.7min
[Parallel(n_jobs=6)]: Done 1238 tasks      | elapsed:  2.7min
[Parallel(n_jobs=6)]: Done 1788 tasks      | elapsed:  3.9min
[Parallel(n_jobs=6)]: Done 2438 tasks      | elapsed:  5.2min
[Parallel(n_jobs=6)]: Done 3188 tasks      | elapsed:  6.9min
[Parallel(n_jobs=6)]: Done 4038 tasks      | elapsed:  8.8min
[Parallel(n_jobs=6)]: Done 4988 tasks      | elapsed: 11.0min
[Parallel(n_jobs=6)]: Done 6038 tasks      | elapsed: 13.5min
[Parallel(n_jobs=6)]: Done 7188 tasks      | elapsed: 16.2min
[Parallel(n_jobs=6)]: Done 8438 tasks      | elapsed: 19.1min
[Parallel(n_jobs=6)]: Done 9788 tasks      | elapsed: 22.4min
[Parallel(n_jobs=6)]: Done 11238 tasks      | elapsed: 25.9mi

In [14]:
# Train Set
train_dfs_sentiment = [x[0] for x in dfs_train if isinstance(x[0], pd.DataFrame)]
train_dfs_metadata = [x[1] for x in dfs_train if isinstance(x[1], pd.DataFrame)]

train_dfs_sentiment  = pd.concat(train_dfs_sentiment, ignore_index=True, sort=False)
train_dfs_metadata = pd.concat(train_dfs_metadata, ignore_index=True, sort=False)
print(train_dfs_sentiment.shape, train_dfs_metadata.shape)

(14442, 6) (58311, 7)


In [15]:
# Test Set
dfs_test = Parallel(n_jobs=6, verbose=1)(delayed(extract_additional_features)(i, mode='test') for i in test_pet_ids)

test_dfs_sentiment = [x[0] for x in dfs_test if isinstance(x[0], pd.DataFrame)]
test_dfs_metadata = [x[1] for x in dfs_test if isinstance(x[1], pd.DataFrame)]

test_dfs_sentiment = pd.concat(test_dfs_sentiment, ignore_index=True, sort=False)
test_dfs_metadata = pd.concat(test_dfs_metadata, ignore_index=True, sort=False)
print(test_dfs_sentiment.shape, test_dfs_metadata.shape)


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    1.8s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    7.5s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:   18.2s
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:   33.4s
[Parallel(n_jobs=6)]: Done 1238 tasks      | elapsed:   52.5s
[Parallel(n_jobs=6)]: Done 1788 tasks      | elapsed:  1.3min
[Parallel(n_jobs=6)]: Done 2438 tasks      | elapsed:  1.8min
[Parallel(n_jobs=6)]: Done 3188 tasks      | elapsed:  2.3min
[Parallel(n_jobs=6)]: Done 3972 out of 3972 | elapsed:  2.8min finished


(3865, 6) (14465, 7)


In [16]:
train_dfs_metadata.head()


,metadata_annots_score,metadata_color_score,metadata_color_pixelfrac,metadata_crop_conf,metadata_crop_importance,metadata_annots_top_desc,PetID
0,0.973658,0.0748377,0.066331,0.8,1,cat black cat,86e1089a3
1,0.926158,0.0873303,0.0681004,0.8,1,cat whiskers small to medium sized cats,6296e909a
2,0.951545,0.0960851,0.0592612,0.8,1,cat small to medium sized cats,6296e909a
3,0.947711,0.0985924,0.0407908,0.8,1,dog dog like mammal dog breed,3422e4906
4,0.95744,0.0983332,0.0969476,0.8,1,dog dog like mammal dog breed,3422e4906


In [17]:
# Extend aggregates and imporve column naming
aggregates =['mean', 'sum']

#Train
train_metadata_desc = train_dfs_metadata.groupby(['PetID'])['metadata_annots_top_desc'].unique()
train_metadata_desc = train_metadata_desc.reset_index()
train_metadata_desc['metadata_annots_top_desc'] = train_metadata_desc['metadata_annots_top_desc'].apply(lambda x: " ".join(x))

prefix = 'metadata'
train_metadata_gr = train_dfs_metadata.drop(['metadata_annots_top_desc'], axis=1)
for i in train_metadata_gr.columns:
    if 'PetID' not in i:
        train_metadata_gr[i] = train_metadata_gr[i].astype(float)

train_metadata_gr = train_metadata_gr.groupby(['PetID']).agg(aggregates)
train_metadata_gr.columns = pd.Index([f'{prefix}_{c[0]}_{c[1].upper()}' for c in train_metadata_gr.columns.tolist()])
train_metadata_gr = train_metadata_gr.reset_index()

train_sentiment_desc = train_dfs_sentiment.groupby(['PetID'])['sentiment_entities'].unique()
train_sentiment_desc = train_sentiment_desc.reset_index()
train_sentiment_desc['sentiment_entities']= train_sentiment_desc['sentiment_entities'].apply(lambda x: " ".join(x))

prefix = 'sentiment'
train_sentiment_gr = train_dfs_sentiment.drop(['sentiment_entities'], axis=1)
for i in train_sentiment_gr.columns:
    if 'PetID' not in i:
        train_sentiment_gr[i] = train_sentiment_gr[i].astype(float)

train_sentiment_gr = train_sentiment_gr.groupby(['PetID']).agg(aggregates)
train_sentiment_gr.columns = pd.Index([f'{prefix}_{c[0]}_{c[1].upper()}' for c in train_sentiment_gr.columns.tolist()])
train_sentiment_gr = train_sentiment_gr.reset_index()

# Test

test_metadata_desc = test_dfs_metadata.groupby(['PetID'])['metadata_annots_top_desc'].unique()
test_metadata_desc = test_metadata_desc.reset_index()
test_metadata_desc['metadata_annots_top_desc'] = test_metadata_desc['metadata_annots_top_desc'].apply(lambda x: " ".join(x))

prefix = 'metadata'
test_metadata_gr = test_dfs_metadata.drop(['metadata_annots_top_desc'], axis=1)
for i in test_metadata_gr.columns:
    if 'PetID' not in i:
        test_metadata_gr[i] = test_metadata_gr[i].astype(float)

test_metadata_gr = test_metadata_gr.groupby(['PetID']).agg(aggregates)
test_metadata_gr.columns = pd.Index([f'{prefix}_{c[0]}_{c[1].upper()}' for c in test_metadata_gr.columns.tolist()])
test_metadata_gr = test_metadata_gr.reset_index()

test_sentiment_desc = test_dfs_sentiment.groupby(['PetID'])['sentiment_entities'].unique()
test_sentiment_desc = test_sentiment_desc.reset_index()
test_sentiment_desc['sentiment_entities']= test_sentiment_desc['sentiment_entities'].apply(lambda x: " ".join(x))

prefix = 'sentiment'
test_sentiment_gr = test_dfs_sentiment.drop(['sentiment_entities'], axis=1)
for i in test_sentiment_gr.columns:
    if 'PetID' not in i:
        test_sentiment_gr[i] = test_sentiment_gr[i].astype(float)

test_sentiment_gr = test_sentiment_gr.groupby(['PetID']).agg(aggregates)
test_sentiment_gr.columns = pd.Index([f'{prefix}_{c[0]}_{c[1].upper()}' for c in test_sentiment_gr.columns.tolist()])
test_sentiment_gr = test_sentiment_gr.reset_index()

In [18]:
# Train Merges:
train_proc = train.copy()
train_proc = train_proc.merge(train_sentiment_gr, how='left', on='PetID')
train_proc = train_proc.merge(train_metadata_gr, how='left', on='PetID')
train_proc = train_proc.merge(train_sentiment_desc, how='left', on='PetID')
train_proc = train_proc.merge(train_metadata_desc, how='left', on='PetID')

# Test Merges:
test_proc = test.copy()
test_proc = test_proc.merge(test_sentiment_gr, how='left', on='PetID')
test_proc = test_proc.merge(test_metadata_gr, how='left', on='PetID')
test_proc = test_proc.merge(test_sentiment_desc, how='left', on='PetID')
test_proc = test_proc.merge(test_metadata_desc, how='left', on='PetID')

print(train_proc.shape, test_proc.shape)
assert train_proc.shape[0] == train.shape[0]
assert test_proc.shape[0] == test.shape[0]

(14993, 44) (3972, 43)


## Breed Mapping


In [19]:
train_breed_main = train_proc[['Breed1']].merge(
                    breed_labels, how='left', left_on='Breed1',
                    right_on='BreedID', suffixes=('', '_main_breed'))

train_breed_main = train_breed_main.iloc[:,2:]
train_breed_main = train_breed_main.add_prefix('main_breed_')

train_breed_second = train_proc[['Breed2']].merge(
                    breed_labels, how='left', left_on='Breed2',
                    right_on='BreedID', suffixes=('', '_second_breed'))

train_breed_second = train_breed_second.iloc[:,2:]
train_breed_second = train_breed_second.add_prefix('second_breed_')

train_proc = pd.concat([train_proc, train_breed_main,train_breed_second], axis=1)

test_breed_main = test_proc[['Breed1']].merge(
                    breed_labels, how='left',
                    left_on='Breed1', right_on='BreedID',
                    suffixes=('','_main_breed')
                    )

test_breed_main = test_breed_main.iloc[:,2:]
test_breed_main = test_breed_main.add_prefix('main_breed_')

test_breed_second = test_proc[['Breed2']].merge(
                    breed_labels, how='left', left_on='Breed2',
                    right_on='BreedID', suffixes=('', '_second_breed'))

test_breed_second = test_breed_second.iloc[:,2:]
test_breed_second = test_breed_second.add_prefix('second_breed_')

test_proc = pd.concat([test_proc, test_breed_main,test_breed_second], axis=1)

print(train_proc.shape, test_proc.shape)


(14993, 48) (3972, 47)


## Concatenate Train and test
### Inspect NaN srtructure

In [20]:
X = pd.concat([train_proc, test_proc], ignore_index=True, sort=False)
print(f'NaN Structure: \n{np.sum(pd.isnull(X))}')

NaN Structure: 
Type                                               0
Name                                            1668
Age                                                0
Breed1                                             0
Breed2                                             0
Gender                                             0
Color1                                             0
Color2                                             0
Color3                                             0
MaturitySize                                       0
FurLength                                          0
Vaccinated                                         0
Dewormed                                           0
Sterilized                                         0
Health                                             0
Quantity                                           0
Fee                                                0
State                                              0
RescuerID                     

In [26]:
column_types[column_types == 'int64']

Type            int64
Age             int64
Breed1          int64
Breed2          int64
Gender          int64
Color1          int64
Color2          int64
Color3          int64
MaturitySize    int64
FurLength       int64
Vaccinated      int64
Dewormed        int64
Sterilized      int64
Health          int64
Quantity        int64
Fee             int64
State           int64
VideoAmt        int64
dtype: object

In [27]:
column_types = X.dtypes
int_cols = column_types[column_types == 'int64']
float_cols = column_types[column_types =='float']
cat_cols = column_types[column_types == 'object']

print(f'\tInteger Columns:\n{int_cols}')
print(f'\n\tFloat Columns:\n{float_cols}')
print(f'\n\Categorical Columns:\n{cat_cols}')

	Integer Columns:
Type            int64
Age             int64
Breed1          int64
Breed2          int64
Gender          int64
Color1          int64
Color2          int64
Color3          int64
MaturitySize    int64
FurLength       int64
Vaccinated      int64
Dewormed        int64
Sterilized      int64
Health          int64
Quantity        int64
Fee             int64
State           int64
VideoAmt        int64
dtype: object

	Float Columns:
PhotoAmt                                       float64
AdoptionSpeed                                  float64
sentiment_sentiment_magnitude_MEAN             float64
sentiment_sentiment_magnitude_SUM              float64
sentiment_sentiment_score_MEAN                 float64
sentiment_sentiment_score_SUM                  float64
sentiment_sentiment_document_magnitude_MEAN    float64
sentiment_sentiment_document_magnitude_SUM     float64
sentiment_sentiment_document_score_MEAN        float64
sentiment_sentiment_document_score_SUM         float64
metad

## Feature Engineering


In [31]:
X_temp = X.copy()

text_columns= ['Description','metadata_annots_top_desc','sentiment_entities']
categorical_columns=['main_breed_BreedName','second_breed_BreedName']

to_drop_columns=['PetID', 'Name','RescuerID']

In [32]:
rescuer_count = X.groupby(['RescuerID'])['PetID'].count().reset_index()
rescuer_count.columns = ['RescuerID','Rescuer_count']

# Merger rescuer count
X_temp = X_temp.merge(rescuer_count, how='left', on='RescuerID')

In [33]:
# Factorize categorical columns:
for i in categorical_columns:
    X_temp.loc[:,i] = pd.factorize(X_temp.loc[:,i])[0]

In [34]:
X_text = X_temp[text_columns]

for i in X_text.columns:
    X_text.loc[:,i] = X_text.loc[:,i].fillna('<MISSING>')

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import SparsePCA, TruncatedSVD, LatentDirichletAllocation, NMF

n_components = 5
text_features=[]

# Generate Text Features
# Decomposition methods
for i in X_text.columns:
    print(f'Generating Features from: {i}')
    svd_ = TruncatedSVD(n_components=n_components, random_state=221)
    nmf_ = NMF(n_components=n_components, random_state=221)
    
    tfidf_col = TfidfVectorizer().fit_transform(X_text.loc[:,i].values)
    svd_col = svd_.fit_transform(tfidf_col)
    svd_col = pd.DataFrame(svd_col)
    svd_col = svd_col.add_prefix(f'SVD_{i}_')
    
    nmf_col = nmf_.fit_transform(tfidf_col)
    nmf_col = pd.DataFrame(nmf_col)
    nmf_col = nmf_col.add_prefix(f'NMF_{i}_')
    
    text_features.append(svd_col)
    text_features.append(nmf_col)
    
# Combine extracted features:
text_features = pd.concat(text_features, axis =1)

# Concatenate with main df:
X_temp = pd.concat([X_temp,text_features], axis=1)

# Remove raw text columns
for i in X_text.columns:
    X_temp = X_temp.drop(i, axis =1)
    

Generating Features from: Description
Generating Features from: metadata_annots_top_desc
Generating Features from: sentiment_entities


In [39]:
# Remove unnecessary columns:
X_temp = X_temp.drop(to_drop_columns, axis=1)

# Check final df shape:
print(f'X shape: {X_temp.shape}')

X shape: (18965, 73)


## Train Test Split

In [43]:
# Split training and test set
X_train = X_temp.loc[np.isfinite(X_temp['AdoptionSpeed']),:]
X_test = X_temp.loc[~np.isfinite(X_temp['AdoptionSpeed']),:]

# Remove missing target column from test:
X_test = X_test.drop(['AdoptionSpeed'], axis=1)

print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
      
assert X_train.shape[0] ==  train.shape[0]
assert X_test.shape[0] == test.shape[0]
      
# Check if columns between the two DFs are the same
train_cols = X_train.columns.tolist()
train_cols.remove('AdoptionSpeed')
      
test_cols = X_test.columns.tolist()
      
assert np.all(train_cols == test_cols)

X_train shape: (14993, 73)
X_test shape: (3972, 72)


In [44]:
np.sum(pd.isnull(X_train))

Type                                             0
Age                                              0
Breed1                                           0
Breed2                                           0
Gender                                           0
Color1                                           0
Color2                                           0
Color3                                           0
MaturitySize                                     0
FurLength                                        0
Vaccinated                                       0
Dewormed                                         0
Sterilized                                       0
Health                                           0
Quantity                                         0
Fee                                              0
State                                            0
VideoAmt                                         0
PhotoAmt                                         0
AdoptionSpeed                  

In [45]:
np.sum(pd.isnull(X_test))

Type                                             0
Age                                              0
Breed1                                           0
Breed2                                           0
Gender                                           0
Color1                                           0
Color2                                           0
Color3                                           0
MaturitySize                                     0
FurLength                                        0
Vaccinated                                       0
Dewormed                                         0
Sterilized                                       0
Health                                           0
Quantity                                         0
Fee                                              0
State                                            0
VideoAmt                                         0
PhotoAmt                                         0
sentiment_sentiment_magnitude_M

## Model Training


In [47]:
import scipy as sp

from collections import Counter
from functools import partial
from math import sqrt

from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.metrics import confusion_matrix as sk_cmatrix

# The following 3 functions have been taken from Ben Hamner's github repository
# https://github.com/benhamner/Metrics

def confusion_matrix(rater_a, rater_b, min_rating=None, max_rating=None):
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)] for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat

def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings

def quadratic_weighted_kappa(y, y_pred):
    """
    Calculates the quadratic weighted kappa
    axquadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = y
    rater_b = y_pred
    min_rating = None
    max_rating = None
    